<a href="https://colab.research.google.com/github/ZIZOUTOUMOU/LAB02-AdvDB/blob/main/tp-hamoud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install openai-whisper pydub
!apt-get install -y ffmpeg



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [12]:
from google.colab import files

uploaded = files.upload()


Saving family guy- stewie cussing out a black guy(deleted scene).mp3 to family guy- stewie cussing out a black guy(deleted scene).mp3


In [14]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.6 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=5448dffe26d380c92729b7b710899dfe33aa50d9516395d35e1d57fea87e19f3
  Stored in directory: /root/.cache/pip/wheels/29/20/ab/f101da7b245b996aa097685ef742243725ea6150f5b3b6d9ed
Successfully built libarchive


In [30]:
# ===== FULL MP3 CENSOR SCRIPT =====
from google.colab import files
import os
import string
import whisper
from pydub import AudioSegment
from pydub.generators import Sine

# ===== STEP 1: Upload MP3 =====
uploaded = files.upload()
filename = next(iter(uploaded))
os.rename(filename, "input.mp3")
input_file = "input.mp3"
output_file = "censored.mp3"

print("Uploaded file saved as input.mp3")

# ===== STEP 2: Define bad words =====
bad_words = [
    "fuck", "fucking",
    "mother fucker", "motherfucker",
    "shit", "bitch", "asshole"  # Add more as needed
]

# ===== STEP 3: Load audio =====
audio = AudioSegment.from_file(input_file)

# ===== STEP 4: Load Whisper model =====
model = whisper.load_model("base")
result = model.transcribe(input_file, word_timestamps=True)
print("✅ Transcription complete")

# ===== STEP 5: Beep function =====
def make_beep(duration_ms):
    return Sine(1000).to_audio_segment(duration=duration_ms).apply_gain(-5)

censored_audio = audio

# ===== STEP 6: Censor loop =====
for segment in result["segments"]:
    words = segment["words"]
    i = 0
    while i < len(words):
        # Clean word: lowercase + remove punctuation
        word = words[i]["word"].strip().lower().translate(str.maketrans('', '', string.punctuation))

        # ---- SINGLE WORD MATCH ----
        if word in bad_words:
            start_ms = int(words[i]["start"] * 1000)
            end_ms = int(words[i]["end"] * 1000)
            beep = make_beep(end_ms - start_ms)
            censored_audio = censored_audio[:start_ms] + beep + censored_audio[end_ms:]
            print(f"Censored single word: {word}")
            i += 1
            continue

        # ---- TWO-WORD PHRASE MATCH ----
        if i < len(words) - 1:
            next_word = words[i+1]["word"].strip().lower().translate(str.maketrans('', '', string.punctuation))
            phrase = f"{word} {next_word}"
            if phrase in bad_words:
                start_ms = int(words[i]["start"] * 1000)
                end_ms = int(words[i+1]["end"] * 1000)
                beep = make_beep(end_ms - start_ms)
                censored_audio = censored_audio[:start_ms] + beep + censored_audio[end_ms:]
                print(f"Censored phrase: {phrase}")
                i += 2
                continue

        i += 1

# ===== STEP 7: Save censored audio =====
censored_audio.export(output_file, format="mp3")
print("✅ Censored audio saved as:", output_file)

# ===== STEP 8: Download censored audio =====
files.download(output_file)


Saving family guy- stewie cussing out a black guy(deleted scene).mp3 to family guy- stewie cussing out a black guy(deleted scene) (1).mp3
Uploaded file saved as input.mp3


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Transcription complete
Censored single word: motherfucker
Censored single word: fuck
Censored single word: fuck
Censored single word: asshole
Censored single word: fuck
Censored single word: fuck
✅ Censored audio saved as: censored.mp3


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>